# 分析テーマ
1. RFM分析
2. メールの開封率予測
3. キャンペーンの効果検証

> コードの詳細な説明はしません．

## データの読み込み
今回使用するデータは以下の4つ．
1. ユーザーデータ: ユーザーIDで一意となっているマスターデータ
2. 購買データ: 購買行動を保存したデータ
3. メールデータ: メールの開封履歴を保存したデータ
4. クーポンデータ: クーポンの配布有無とその後1週間の購買活動を保存したデータ

### 1. ユーザーデータ
2つのcsvファイル
- user_master_2022: 2022年までのマスターデータ
- user_master_2023: 2023年1月7日までのマスターデータ（今回は使用しない）

カラムの概要

|カラム名|概要|データ型|備考|
|-|-|-|-|
|id|ユーザーID|int|ユーザーを一意に識別（PK）|
|gender|性別|string|男性or女性|
|age|年齢|int|特になし|
|first_purchase_date|初回購買日|string|文字列型→日付型の変換が必要|
|last_purchase_date|最終購買日|string|文字列型→日付型の変換が必要|

### 2. 購買データ
purchase_master_2022.csv

カラムの概要

|カラム名|概要|データ型|備考|
|-|-|-|-|
|id|ユーザーID|int|ユーザーを一意に識別|
|purchase_price|購買金額|int|特になし|
|purchase_date|購買日|string|文字列型→日付型の変換が必要|

### 3. メールデータ
mail_master_202211.csv

カラムの概要

|カラム名|概要|データ型|備考|
|-|-|-|-|
|id|ユーザーID|int|ユーザーを一意に識別（PK）|
|mail_opened|メール開封ダミー|int|開封1, 未開封0|

### 4. クーポンデータ
coupon_master_202301.csv

カラムの概要

|カラム名|概要|データ型|備考|
|-|-|-|-|
|id|ユーザーID|int|ユーザーを一意に識別（PK）|
|coupon_dummy|クーポン配布ダミー|int|配布1, 未配布0|
|purchase_dummy|購買ダミー|int|クーポン配布日以降に購買あり1, 購買なし0|
|purchase_price|購買金額|int|クーポン配布日以降の購買金額|

In [1]:
# 必要なライブラリのインポート
import pandas as pd

# データのPATHを格納
user_path = 'https://raw.githubusercontent.com/s1ok69oo/LT_rep/main/RareTECH_lecture_20230607/data/user_master_2022.csv'
purchase_path = 'https://raw.githubusercontent.com/s1ok69oo/LT_rep/main/RareTECH_lecture_20230607/data/purchase_master_2022.csv'
mail_path = 'https://raw.githubusercontent.com/s1ok69oo/LT_rep/main/RareTECH_lecture_20230607/data/mail_master_202211.csv'
coupon_path = 'https://raw.githubusercontent.com/s1ok69oo/LT_rep/main/RareTECH_lecture_20230607/data/coupon_master_202301.csv'

# 各データをデータフレームに格納
df_user = pd.read_csv(user_path)
df_purchase = pd.read_csv(purchase_path)
df_mail = pd.read_csv(mail_path)
df_coupon = pd.read_csv(coupon_path)

# ユーザーデータの中身を確認
print('--------------------')
print('ユーザーデータ')
print('--------------------')
display(df_user.head())

# 購買データの中身を確認
print('--------------------')
print('購買データ')
print('--------------------')
display(df_purchase.head())

# メールデータの中身を確認
print('--------------------')
print('メールデータ')
print('--------------------')
display(df_mail.head())

# クーポンデータの中身を確認
print('--------------------')
print('クーポンデータ')
print('--------------------')
display(df_coupon.head())

--------------------
ユーザーデータ
--------------------


,id,gender,age,first_purchase_date,last_purchase_date
0,1,女性,50,2021-04-03,2022-01-20
1,2,男性,65,2021-04-06,2022-01-25
2,3,女性,40,2021-04-18,2022-01-16
3,4,女性,52,2021-04-19,2022-01-10
4,5,女性,54,2021-05-12,2022-01-13


--------------------
購買データ
--------------------


,id,purchase_price,purchase_date
0,1,10000,2021-04-03
1,2,10000,2021-04-06
2,1,3000,2021-04-16
3,3,3000,2021-04-18
4,4,5000,2021-04-19


--------------------
メールデータ
--------------------


,id,mail_opened
0,1,1
1,2,1
2,3,1
3,4,0
4,5,1


--------------------
クーポンデータ
--------------------


,id,coupon_dummy,purchase_dummy,purchase_price
0,1,1,1,3000
1,2,1,1,5000
2,3,1,1,3000
3,4,1,1,3000
4,5,1,0,0


# RFM分析
4つのセグメントに分割し，各セグメントのIDリストを出力する．

|セグメント|R|F・M|
|-|-|-|
|優良顧客|小|大|
|新規・特定利用顧客|小|小|
|過去の優良顧客|大|大|
|優先度の低い顧客|小|小|


In [2]:
# 必要なライブラリをインポート
import datetime
import pandas as pd

# 必要なデータを再度データフレームに格納（一度やっていれば不要）
df_purchase = pd.read_csv(purchase_path)

# 日付カラムが文字列データとなっているので，日付データに変換（一度やっていれば不要）
df_purchase['purchase_date'] = pd.to_datetime(df_purchase['purchase_date'])

# IDベースでデータを集計
df = df_purchase.groupby('id', as_index=False).agg(rc=('purchase_date', 'max'), 
                                                   fq=('purchase_date', 'count'), 
                                                   mo=('purchase_price', 'sum'))

# recencyを評価しやすくするために，rcカラムを2022年12月31日との差分に変更
base_date = datetime.datetime(2022, 12, 31)
df['rc'] = df['rc'].apply(lambda x: base_date-x).dt.days

# 各セグメントの基準を設定
rc_median = df['rc'].median()
fq_median = df['fq'].median()
mo_median = df['mo'].median()

# 優良顧客（R小, F・M大）
df1 = df[(df['rc']<rc_median)&(df['fq']>fq_median)&(df['mo']>mo_median)]
# 新規顧客・特定利用顧客（R小. F・M小）
df2 = df[(df['rc']<rc_median)&(df['fq']<fq_median)&(df['mo']<mo_median)]
# 過去の優良顧客（R大, F・M大）
df3 = df[(df['rc']>rc_median)&(df['fq']>fq_median)&(df['mo']>mo_median)]
# 優先度の低い顧客（R大, F・M小）
df4 = df[(df['rc']>rc_median)&(df['fq']<fq_median)&(df['mo']<mo_median)]

# それぞれのIDをリストに格納
id1_lst = df1['id'].to_list()
id2_lst = df2['id'].to_list()
id3_lst = df3['id'].to_list()
id4_lst = df4['id'].to_list()

# IDリストを出力
print(f"ID1リスト: {id1_lst}")
print(f"ID2リスト: {id2_lst}")
print(f"ID3リスト: {id3_lst}")
print(f"ID4リスト: {id4_lst}")

ID1リスト: [15, 17, 29, 35, 38, 43, 46, 47, 50, 56, 59, 60, 61, 84, 87, 88, 89, 98, 102, 103, 104, 107, 113, 116, 119, 120, 123, 124, 128, 133, 136, 137, 139, 141, 142, 143, 145, 147, 155, 158, 159, 160, 161, 162, 164, 165, 167, 172, 176, 177, 180, 182, 183, 185, 189, 191, 192, 193, 196, 198, 199, 200, 201, 202, 203, 205, 206, 207, 218, 219, 220, 221, 223, 225, 230, 231, 235, 238, 239, 242, 243, 245, 249, 250, 253, 256, 259, 266, 269, 276, 277, 278, 279, 283, 285, 292, 293, 295, 297, 299, 306, 307, 313, 314, 316, 317, 321, 322, 326, 328, 329, 330, 333, 336, 338, 341, 344, 347, 348, 349, 356, 357, 358, 360, 361, 362, 363, 365, 366, 368, 369, 370, 371, 374, 380, 385, 387, 388, 390, 391, 396, 398, 399, 401, 403, 407, 408, 409, 412, 413, 414, 416, 418, 419, 420, 421, 422, 424, 425, 426, 429, 431, 433, 436, 448, 450, 451, 453, 456, 463, 464, 472, 475, 476, 480, 481, 482, 484, 485, 486, 487, 489, 492, 495, 496, 497, 509, 511, 514, 523, 528, 529, 532, 534, 541, 546, 547, 559, 560, 606, 614, 630,

# メールの開封率予測
メールを開封すると予測されたユーザーIDをリストとして出力する

手順
1. データを準備
2. 訓練データとテストデータに分割
3. モデルの選定
4. 訓練データを学習
5. モデルの精度を評価
6. モデルを利用してメール送信ユーザーリストを出力

### 1. データの準備

In [3]:
# 必要なライブラリをインポート
import datetime
import pandas as pd

# 必要なデータを再度データフレームに格納（一度やっていれば不要）
df_user = pd.read_csv(user_path)
df_purchase = pd.read_csv(purchase_path)
df_mail = pd.read_csv(mail_path)

# 日付カラムが文字列データとなっているので，日付データに変換（一度やっていれば不要）
df_user['first_purchase_date'] = pd.to_datetime(df_user['first_purchase_date'])
df_user['last_purchase_date'] = pd.to_datetime(df_user['last_purchase_date'])
df_purchase['purchase_date'] = pd.to_datetime(df_purchase['purchase_date'])

# メール送信日（2022年11月1日）を格納
send_date = datetime.datetime(2022, 11, 1)

# ユーザーデータをメール送信日以前のものに絞り込む
df_user_202211 = df_user[df_user['first_purchase_date']<send_date]
df_user_202211 = df_user_202211[['id', 'gender', 'age']]

# 購買データをメール送信日以前のものに絞り込んでIDベースで集計する
df_purchase_202211 = df_purchase[df_purchase['purchase_date']<send_date]
df_purchase_202211 = df_purchase_202211.groupby('id', as_index=False).agg(rc=('purchase_date', 'max'), 
                                                                          fq=('purchase_date', 'count'), 
                                                                          mo=('purchase_price', 'sum'))

# データを結合
## ユーザーデータと購買データを結合
df = df_user_202211.merge(df_purchase_202211, how='left', on='id')
## 上記のデータとメールデータを結合
df = df.merge(df_mail, how='left', on='id')

# モデル構築のためのデータの前処理
## genderカラムを数値データ（男性1, 女性0）に変換
df['gender'] = df['gender'].apply(lambda x: 1 if x=='男性' else 0)
## frequencyカラムを生成
df['rc'] = df['rc'].apply(lambda x: send_date-x).dt.days
## 各特徴量（age, recency, frequency, monetary）を正規化
df['age'] = (df['age'].max()-df['age']) / df['age'].max()
df['rc'] = (df['rc'].max()-df['rc']) / df['rc'].max()
df['fq'] = (df['fq'].max()-df['fq']) / df['fq'].max()
df['mo'] = (df['mo'].max()-df['mo']) / df['mo'].max()

# 目的変数と特徴量の設定
X = df[['gender', 'age', 'rc', 'fq', 'mo']]
y = df['mail_opened']

# 特徴量の確認
display(X.head())
print('---------------------')
print('メール開封ダミー')
print('---------------------')
display(y.head())

,gender,age,rc,fq,mo
0,0,0.253731,0.046823,0.347826,0.388889
1,1,0.029851,0.063545,0.347826,0.375000
2,0,0.402985,0.033445,0.391304,0.548611
3,0,0.223881,0.013378,0.434783,0.479167
4,0,0.194030,0.023411,0.391304,0.423611


---------------------
メール開封ダミー
---------------------


0    1
1    1
2    1
3    0
4    1
Name: mail_opened, dtype: int64

### 2. テストデータと訓練データに分割する

In [4]:
# 必要なライブラリのインポート
from sklearn.model_selection import train_test_split

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

### 3・4. モデルの選定および訓練データの学習
今回は「ランダム・フォレスト」というモデルを使用

> 今回はパラメータの調整等は行いません

In [5]:
# 必要なライブラリをインポート
from sklearn.ensemble import RandomForestClassifier

# 訓練データの学習
clf = RandomForestClassifier(random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

### 5. モデルの評価
指標の確認

||陽性（予測）|陰性（予測）|
|-|-|-|
|陽性（実際）|真陽性（TP）|偽陰性（FN）|
|陰性（実際）|偽陽性（FP）|真陰性（TN）|

> T: True, F: False, P: Positive, N: Negative

- 正解率（accuracy）$$\frac{TP+TN}{TP+FP+FN+TN}$$
- 適合率（precision）$$\frac{TP}{TP+FP}$$
- 再現率（recall）$$\frac{TP}{TP+FN}$$
- F値$$\frac{2}{\frac{1}{precision}+\frac{1}{recall}}$$

In [6]:
# 必要なライブラリをインポート
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# テストデータを用いて予測値を算出
y_pred = clf.predict(X_test)

# モデルの評価
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"正解率: {accuracy*100:.1f}%")
print(f"適合率: {precision*100:.1f}%")
print(f"再現率: {recall*100:.1f}%")
print(f"F値: {f1*100:.1f}%")

正解率: 58.7%
適合率: 54.2%
再現率: 59.8%
F値: 56.9%


### モデルを利用してメール送信ユーザーリストを出力

In [8]:
# 実運用
send_date = datetime.datetime(2022, 12, 31)
df_user_202212 = df_user[['id', 'gender', 'age']]
df_purchase_202212 = df_purchase.groupby('id', as_index=False).agg(rc=('purchase_date', 'max'), 
                                                                   fq=('purchase_date', 'count'), 
                                                                   mo=('purchase_price', 'sum'))
# ユーザーデータと購買データを結合
df = df_user_202212.merge(df_purchase_202212, how='left', on='id')

# モデル構築のためのデータの前処理
## genderカラムを数値データ（男性1, 女性0）に変換
df['gender'] = df['gender'].apply(lambda x: 1 if x=='男性' else 0)
## frequencyカラムを生成
df['rc'] = df['rc'].apply(lambda x: send_date-x).dt.days

## 各特徴量の正規化
df['age'] = (df['age'].max()-df['age']) / df['age'].max()
df['rc'] = (df['rc'].max()-df['rc']) / df['rc'].max()
df['fq'] = (df['fq'].max()-df['fq']) / df['fq'].max()
df['mo'] = (df['mo'].max()-df['mo']) / df['mo'].max()

## 特徴量の設定
X = df[['gender', 'age', 'rc', 'fq', 'mo']]

# 予測
y_pred = clf.predict(X)
# 予測結果の格納
df['y_pred'] = y_pred

# メールを開封すると予測されたユーザーIDを出力
send_lst = df[df['y_pred']==1]['id'].tolist()
print(send_lst)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 16, 18, 20, 21, 22, 23, 27, 28, 29, 31, 32, 33, 34, 35, 37, 38, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 53, 56, 57, 58, 59, 61, 67, 68, 70, 71, 74, 76, 80, 84, 87, 88, 91, 93, 95, 100, 101, 102, 103, 104, 105, 107, 108, 112, 113, 116, 117, 119, 120, 121, 122, 123, 126, 127, 128, 132, 133, 134, 136, 137, 139, 141, 142, 143, 147, 152, 155, 158, 159, 161, 162, 163, 164, 165, 167, 170, 171, 176, 177, 178, 180, 182, 183, 185, 186, 188, 189, 191, 192, 193, 196, 197, 198, 200, 202, 203, 205, 206, 207, 214, 218, 219, 220, 223, 224, 225, 226, 230, 231, 233, 234, 235, 236, 238, 241, 242, 244, 245, 249, 250, 253, 256, 257, 258, 259, 263, 265, 266, 270, 276, 278, 279, 283, 285, 289, 292, 293, 295, 297, 299, 300, 302, 303, 305, 306, 307, 309, 313, 317, 321, 322, 324, 325, 328, 329, 330, 333, 334, 335, 336, 341, 344, 345, 347, 348, 349, 356, 357, 358, 359, 360, 361, 362, 366, 368, 369, 374, 383, 385, 387, 388, 390, 397, 398, 399, 401, 405, 406, 407, 408, 4

## 効果検証
2つのグループの購買率を比較する
- 最終購買日時が91~100日の（クーポンを配布された）ユーザー
- 最終購買日時が81z~90日の（クーポンを配布されていない）ユーザー

Z検定と呼ばれる検定（統計学的手法）を実施

In [9]:
# 必要なライブラリをインポート（一度インポートしていれば不要）
import pandas as pd

# 必要なデータを再度データフレームに格納（一度やっていれば不要）
df_user = pd.read_csv(user_path)
df_coupon = pd.read_csv(coupon_path)

# 日付カラムが文字列データとなっているので，日付データに変換（一度やっていれば不要）
df_user['first_purchase_date'] = pd.to_datetime(df_user['first_purchase_date'])
df_user['last_purchase_date'] = pd.to_datetime(df_user['last_purchase_date'])

# ユーザーデータとクーポンデータを結合
df = df_user.merge(df_coupon, how='left', on='id')

# 最終購買日時が90日±10日間のデータを抽出
cutoff_date = datetime.datetime(2022, 10, 1)
td = datetime.timedelta(days=10)
lower_date = cutoff_date - td
upper_date = cutoff_date + td
df= df[(df['last_purchase_date']>lower_date)&(df['last_purchase_date']<upper_date)]

# 最終購買日時が90日±10日間のデータの購入率を算出
df[['coupon_dummy', 'purchase_dummy']].groupby('coupon_dummy').mean()

,purchase_dummy
coupon_dummy,
0,0.270270
1,0.444444


In [20]:
# Z検定（比率の差の検定）
from statsmodels.stats.proportion import proportions_ztest

# 該当ユーザー数と購買者数を
df_grouped = df.groupby(by=['coupon_dummy']).agg(n=('coupon_dummy', 'count'), 
                                                 purchase_n=('purchase_dummy', 'sum'))
# 該当ユーザー数
n_lst = df_grouped['n'].to_list()
# 購買者数
purchase_n_lst = df_grouped['purchase_n'].to_list() 

# Z検定の実施
proportions_ztest(purchase_n_lst, n_lst, alternative='smaller')

(-1.0184642420180552, 0.15422869290346664)